# Scraping de datos de SICOIN (2020)

SICOIN es el sistema de información de contabilidad del gob. de Guatemala. Es un sitio web hecho en ASP, aparentemente. Tiene muchos reportes sobre ingresos, gastos y el movimiento del dinero en el estado. Como base de datos, es un sistema muy completo, aunque en realidad no dice gran cosa sobre el gasto público. Es muy sencillo para los funcionarios de Guatemala esconder la corrupción y el despilfarro en esos datos porque son datos ambiguos, con categorías amplias de gasto.  Aunque creo que así es la contabilidad en general: fácilmente se puede mentir en las cuentas.

Como base de datos es aceptable pero la aplicación web es de lo peor. En primer lugar, no sería necesario hacer scrape de estos datos si SICOIN tuviera una API. Pero no, en lugar de API lo que tiene es frames dentro de frames dentro de frames. Es lo que pasa cuando se trabaja con ASP y .Net y esas cosas (aunque he visto apps decentes con esas tecnologías y esta parece estar hecha a propósito de la peor manera). 

Para hacer el scraping uso [Selenium ](https://www.selenium.dev/) con el driver para Google Chrome. He puesto las funciones para conectar con SICOIN en un archivo aparte para organizar mejor las cosas. Al ver el código verán el desastre que tiene SICOIN para acceder al menú principal o a los formularios.

<hr>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import os
from SICOIN_scraper import *
import xlrd
import re
import pandas as pd
import time

In [5]:
options = webdriver.ChromeOptions()
options.headless = False # True
# options.add_argument("download.default_directory="+os.getcwd())
options.add_experimental_option("prefs", {
  "download.default_directory": os.path.abspath("/tmp/sicoin/"),
  "download.prompt_for_download": False,
})

driver = webdriver.Chrome(chrome_options=options, executable_path="../../software/chromedriver")

driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'})


<ipython-input-5-3104707cb48e>:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path="../../software/chromedriver")


{}

In [6]:
driver.get('https://sicoin.minfin.gob.gt')

In [7]:
login(driver)
#driver.close()

In [8]:

# Reporte de gastos por grupos dinámicos
clicks_sequence = ["itemTextLink32", "itemTextLink50", "itemTextLink51", "itemTextLink55"]
def set_grupos():
        driver.find_element_by_id("lstCorte").send_keys("PROGRA")
        driver.find_element_by_id("lstColumna").send_keys("ACTIVIDAD U")
    

In [15]:
driver.refresh()

In [14]:
for year in range(2020,2021):
    print(year)
    setYear(driver, year)
    time.sleep(2)
    reportTypeA(driver, clicks_sequence, [], other_options=set_grupos)
    time.sleep(10)
    driver.refresh()

2020
downloading


# Parse data

In [18]:
# Read all excel files:
xls_files = []
for root, dirs, files in os.walk("/tmp/sicoin/"):
    n = len(files)
    for i, file in enumerate(files):
        if (file.endswith(".xls") or file.endswith(".xlsx")):
            print(i, file)
            xls_files.append(xlrd.open_workbook(root + "/" + file))

0 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo235554.xls
1 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo233830.xls
2 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo235759.xls
3 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo234333.xls
4 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo233954.xls
5 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo235502.xls
6 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo235838.xls
7 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo235253.xls
8 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo235033.xls
9 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo234211.xls
10 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo235720.xls
11 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo234645.xls
12 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo234509.xls
13 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo235916.xls
14 _172.25.3.18reportes$00804768jjp3w1gtgn5fkwy2xbtqxjvo002545.xls
15 _1

In [124]:
def nextCell(row, offset = 0):
    lenr = len(row)
    if offset >= lenr:
        return None
    while (offset+1 < lenr) & (row[offset].ctype == xlrd.XL_CELL_EMPTY):
        offset += 1
    return offset

def process_row(row, truncate = None):
    rowData = []
    js = []
    j = nextCell(row, 0)
    while True:
        if j is not None:
            if (truncate is not None) and (truncate < j):
                break
            value = str(row[j].value).strip()
            js.append(j)
            rowData.append(value)
        if j is None:
            break    
        j = nextCell(row, j+1)
        
    return rowData, js

def tryFun(fun, param1):
    try:
        return fun(param1)
    except:
        return None

def process_table_simple(sheet, options):
    start = options["start"]
    if (type(start) == dict):
        i = 0
        # Hay que buscar la primera aparición de start["value"] en la columna start["col"] 
        while i<sheet.nrows:
            r = sheet.row(i)
            if (len(r)>start["col"]) and (r[start["col"]].value == start["value"]):
                break
            i += 1
            
    elif type(start) == int:
        i = start
    data = []
    while True:
        processTitleRow = False
        popCategory = False
        i += 1
        if i >= sheet.nrows:
            break
        row = sheet.row(i)
        values, js = process_row(row, options["truncate"] if "truncate" in options else None)
        
        if len(values)==0:
            continue
        if values[0].lower().startswith("tota"):
            continue
        rowData = dict(zip(js, values))
        data.append(rowData)
    df = pd.DataFrame.from_records(data)
    return df

def process_reporte(s):
    df = process_table_simple(s, {"start": 19, "truncate": 45})
    df = df.sort_index(1)
    df.loc[df[2].notna(), 2] = df.loc[df[2].notna(), 2] + " - " + df.loc[df[2].notna(), 10]
    df.drop(columns=44, inplace=True)
    try:
        df.columns = ["CodEntidad", "CodPrograma", "CodActOb","Programa", "ActOb",
                      "Asignado", "Modificado", "Vigente", "PreCompromiso", "Comprometido", "Devengado", 
                      "Pagado", "SaldoPorComprometer", "SaldoPorDevengar", "SaldoPorPagar", "PorcentajeE"]
    except:
        print("Error al poner columnas")
        print(df.head())
        raise ValueError
    for col in ["CodEntidad", "CodPrograma", "CodActOb","Programa", "ActOb"]:
        df[col] = df[col].fillna(method="ffill")
    return df

In [65]:
s = xls_files[0].sheet_by_index(0)

In [66]:
for i in range(0,20):
    print(i, "| ", "  |  ".join([str(x.value)[0:10] for x in s.row(i)]))

0 |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |  
1 |    |    |  Sistema de  |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |  
2 |    |    |  Ejecución   |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |  
3 |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |    |  PAGINA   :  |    |    |  1.0  |    |  DE  |    |  56.0  |    |    |    |  
4 |    |    |  Ejecucion   |    |    |    |    |    |    

In [126]:
# Optimizando código para extraer datos
s = xls_files[0].sheet_by_index(0)
%prun process_reporte(s)

         455853 function calls (455738 primitive calls) in 0.268 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   158400    0.081    0.000    0.111    0.000 sheet.py:403(cell)
     3520    0.036    0.000    0.147    0.000 sheet.py:474(<listcomp>)
    20480    0.033    0.000    0.035    0.000 <ipython-input-124-27a67d165287>:1(nextCell)
     3520    0.030    0.000    0.071    0.000 <ipython-input-124-27a67d165287>:9(process_row)
   158400    0.030    0.000    0.030    0.000 sheet.py:2303(__init__)
        1    0.019    0.019    0.255    0.255 <ipython-input-124-27a67d165287>:32(process_table_simple)
     3520    0.004    0.000    0.151    0.000 sheet.py:470(row)
    37318    0.004    0.000    0.004    0.000 {method 'append' of 'list' objects}
27987/27893    0.004    0.000    0.004    0.000 {built-in method builtins.len}
    16960    0.002    0.000    0.002    0.000 {method 'strip' of 'str' objects}
       21    0.002    0.

In [134]:
# Probando
df = process_reporte(s)

In [135]:
#
df[df.Asignado.notna()].head()

,CodEntidad,CodPrograma,CodActOb,Programa,ActOb,Asignado,Modificado,Vigente,PreCompromiso,Comprometido,Devengado,Pagado,SaldoPorComprometer,SaldoPorDevengar,SaldoPorPagar,PorcentajeE
2,11130003 - PRESIDENCIA DE LA REPÚBLICA,11,11 00 000 001 000,GESTIÓN GUBERNAMENTAL,DESPACHO PRESIDENCIAL,8113671.0,-1927157.0,6186514.0,0.0,6177429.51,6177429.51,6150508.51,9084.49,9084.49,26921.0,99.85315655957459
7,11130003 - PRESIDENCIA DE LA REPÚBLICA,12,12 00 000 001 000,GESTIÓN VICEPRESIDENCIAL,DESPACHO VICEPRESIDENCIAL,23034990.0,-1002969.0,22032021.0,0.0,15050230.76,15050230.76,14935931.17,6981790.24,6981790.24,114299.59,68.31071357457402
12,11130003 - PRESIDENCIA DE LA REPÚBLICA,13,13 00 000 001 000,ASUNTOS ADMINISTRATIVOS Y DE SEGURIDAD,SERVICIOS ADMINISTRATIVOS Y DE SEGURIDAD,156088079.0,1422802.0,157510881.0,0.0,157386954.31,157386954.31,156075517.87,123926.69,123926.69,1311436.44,99.92132182283966
17,11130003 - PRESIDENCIA DE LA REPÚBLICA,14,14 00 000 001 000,SEGURIDAD PERIMETRAL,SEGURIDAD MOVIL PERMANENTE,17950619.0,0.0,17950619.0,0.0,17929045.37,17929045.37,17419397.91,21573.63,21573.63,509647.46,99.87981679071903
22,11130003 - PRESIDENCIA DE LA REPÚBLICA,17,17 00 000 001 000,EMBAJADA EXTRAORDINARIA ITINERANTE DE ASUNTOS ...,DIRECCIÓN Y COORDINACIÓN,1179400.0,-1179400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [136]:
dfs = pd.DataFrame()
files = {}

In [138]:
# Para todos los archivos:
for i, x in enumerate(xls_files):
#try: 
    s = x.sheet_by_index(0)
    year =  0

    year    = s.row(14)[8].value
    print("Procesando ", year)
    # check duplicados
    if (year in files):
        print("se encontró un duplicado", year)
        continue
    files[year] = True
    temp = process_reporte(s)
    if temp is None:
        print(i, "Empty report")
    else:
        temp["Year"] = year
        dfs = dfs.append(temp, ignore_index = True)

Procesando  2014.0
se encontró un duplicado 2014.0
Procesando  2005.0
Procesando  2017.0
Procesando  2008.0
Procesando  2006.0
Procesando  2013.0
Procesando  2018.0
Procesando  2012.0
Procesando  2011.0
Procesando  2007.0
Procesando  2016.0
Procesando  2010.0
Procesando  2009.0
Procesando  2019.0
Procesando  2020.0
Procesando  2015.0
Procesando  2004.0


In [148]:
dfs = dfs[dfs.Asignado.notna()]

In [149]:
dfs.shape

(16573, 17)

In [154]:
#dfs.to_csv("sicoin_programas_acts_2004-2020.csv")
dfs.to_csv("sicoin_programas_acts_2004-2020.csv.gz")

In [155]:
!ls -lha 

total 5.5M
drwxrwxr-x 4 guillermo guillermo  4.0K nov 23 22:55  .
drwxrwxr-x 6 guillermo guillermo  4.0K nov 22 15:56  ..
-rw-rw-r-- 1 guillermo guillermo   29K nov 23 22:55 'explorar datos.ipynb'
drwxrwxr-x 2 guillermo guillermo  4.0K nov 23 22:31  .ipynb_checkpoints
drwxrwxr-x 2 guillermo guillermo  4.0K nov 22 23:36  __pycache__
-rw-rw-r-- 1 guillermo guillermo   75K nov 23 22:43  scrape_presupuestos.ipynb
-rw-rw-r-- 1 guillermo guillermo  4.4M nov 23 22:43  sicoin_programas_acts_2004-2020.csv
-rw-rw-r-- 1 guillermo guillermo 1006K nov 23 22:57  sicoin_programas_acts_2004-2020.csv.gz
-rw-rw-r-- 1 guillermo guillermo  4.1K nov 22 16:47  SICOIN_scraper.py
